# How to use LangChain and Azure OpenAI with Python


Langchain is an open source framework for developing applications using large language models (LLM). <br>

This guide will demonstrate how to setup and use Azure OpenAI models' API with LangChain.
    

## Set Up
The following libraries must be installed to use LangChain with Azure OpenAI.<br>

In [3]:
#!pip install --upgrade openai
#!pip install langchain

## API Configuation and Deployed Model Setup

After installing the necessary libraies, the API must be configured. The code below shows how to configure the API directly in your Python environment. 


In [4]:
import openai
import json
import os
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage
from langchain import LLMChain

In [5]:
# Load config values
with open(r'config.json') as config_file:
    config_details = json.load(config_file)

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai_api_base=config_details['OPENAI_API_BASE']
    
# API version e.g. "2023-07-01-preview"
openai_api_version=config_details['OPENAI_API_VERSION']

# The name of your Azure OpenAI deployment chat model. e.g. "gpt-35-turbo-0613"
deployment_name=config_details['DEPLOYMENT_NAME']

# The API key for your Azure OpenAI resource.
os.environ["OPENAI_API_KEY"] = "62855d6dd08945819bf83aee0c104127"
openai_api_key = os.getenv("OPENAI_API_KEY")

# This is set to `azure`
openai_api_type="azure"

## Deployed Model Setup

In [12]:
# Create an instance of chat llm
#llm = AzureChatOpenAI(openai_api_base=openai_api_base, openai_api_version=openai_api_version,deployment_name=deployment_name,openai_api_key=openai_api_key,openai_api_type=openai_api_type,)

#llm.invoke([HumanMessage(content="Write me a poem")])

In [13]:
# Create an instance of chat llm
llm = AzureChatOpenAI(
    azure_deployment=deployment_name,
    openai_api_version=openai_api_version,
    azure_endpoint=openai_api_base
)

response = llm.invoke([HumanMessage(content="Write me a poem")])

In [15]:
print(response.content)

In the realm of dreams, where words take flight,
Let me weave a poem, shimmering with delight.
A tapestry spun with threads of emotion and thought,
A symphony of words, a canvas of the heart.

On the wings of imagination, let us soar,
To lands unknown, yet familiar to the core.
Where verses dance like fireflies in twilight's glow,
And whispers of the soul softly start to flow.

In this enchanted realm, let beauty unfurl,
Through valleys of laughter and mountains of pearls.
Where the sun paints the sky in hues of gold,
And the moon weaves stories, legends of old.

Let me paint you a picture with words so true,
Of love's tender touch and skies of azure blue.
Of the gentle rain that quenches the thirst,
And the winds that carry secrets, whispered and rehearsed.

Let me capture the essence of a moment in time,
In verses that rhyme, a rhythm sublime.
For poetry, dear friend, is the language of the soul,
A portal to emotions, where hearts become whole.

So let me write a poem, just for you,


## PromptTemplates

Langchain provides a built in PromptsTemplate module to simplify the construction of prompts to get more specific answers.

In [16]:
from langchain import PromptTemplate

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

In [20]:
# First Example
template = """
You are a skin care consulant that recommends products based on customer
needs and preferences.

What is a good {product_type} to help with {customer_request}?
"""

prompt = PromptTemplate(
    input_variables=["product_type", "customer_request"],
    template=template,
)

In [21]:
print("Example #1:")
response = llm.invoke([HumanMessage(content=prompt.format(
        product_type="face wash",
        customer_request = "acne prone skin"
    ))]
)

print(response.content)

Example #1:
A good face wash for acne-prone skin is one that effectively cleanses the skin, removes excess oil, unclogs pores, and helps to prevent future breakouts. Here are a few recommendations:

1. Neutrogena Oil-Free Acne Wash: This face wash contains salicylic acid, which helps to exfoliate the skin and clear acne. It also helps in reducing inflammation and redness caused by acne.

2. La Roche-Posay Effaclar Medicated Gel Cleanser: This gel cleanser is specifically formulated for oily and acne-prone skin. It contains salicylic acid and LHA, which help to unclog pores and reduce excess oil production.

3. CeraVe Acne Foaming Cream Cleanser: This cleanser contains benzoyl peroxide, which is effective in treating acne by killing the bacteria that causes breakouts. It also helps in reducing excess oil and improving skin texture.

4. Mario Badescu Acne Facial Cleanser: This face wash contains salicylic acid and aloe vera, which work together to reduce acne and soothe the skin. It also

In [22]:
# Second Example
system_message = "You are an AI travel assistant that provides vacation recommendations."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_message)

human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chain = LLMChain(llm=llm, prompt=chat_prompt)

In [23]:
result = chain.run(f"Where should I go on vaction in Decemember for warm weather and beaches?")
print("Example #2:")
print(result)

Example #2:
For warm weather and beautiful beaches in December, I would recommend the following destinations:

1. Maldives: This tropical paradise offers pristine white-sand beaches, crystal-clear turquoise waters, and luxurious resorts. The Maldives experiences year-round warm temperatures, making it an ideal destination for beach lovers.

2. Hawaii, USA: With its stunning beaches, lush landscapes, and diverse marine life, Hawaii is a fantastic choice. The islands of Oahu, Maui, and Kauai offer a mix of relaxation and adventure, along with plenty of warm, sunny days.

3. Thailand: Head to the southern islands of Thailand, such as Phuket, Krabi, or Koh Samui, for a beach getaway. These destinations boast stunning beaches, warm waters, and a vibrant local culture. December is also a great time to explore Thailand's stunning national parks and enjoy water activities like snorkeling and diving.

4. Costa Rica: With its beautiful coastline on both the Pacific Ocean and the Caribbean Sea, C

## Chains
There are many applications of chains that allow you to combine numerous LLM calls and actions.  <br>

### Simple Sequential Chains  <br>
Allow you to feed the output of one LLM Chain as input for another.

In [27]:
from langchain.chains import SimpleSequentialChain

In [28]:
description_template = """Your job is to come up with a fun DIY project for the specified gender, age, and description of a kid.
% CHILD_DESCRIPTION
{child_description}

YOUR RESPONSE:
"""
description_prompt_template = PromptTemplate(input_variables=["child_description"], template=description_template)

topic_chain = LLMChain(llm=llm, prompt=description_prompt_template)

In [45]:
topic_chain

LLMChain(prompt=PromptTemplate(input_variables=['child_description'], template='Your job is to come up with a fun DIY project for the specified gender, age, and description of a kid.\n% CHILD_DESCRIPTION\n{child_description}\n\nYOUR RESPONSE:\n'), llm=AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001D1E1A0B8D0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001D1E1A74110>, openai_api_key='62855d6dd08945819bf83aee0c104127', openai_proxy='', azure_endpoint='https://dskumar.openai.azure.com/', deployment_name='DskumarDeployment', openai_api_version='2023-07-01-preview', openai_api_type='azure'))

In [30]:
diy_description_template = """Given a DIY project, give a short and simple recipe step-by-step guide on how to complete the project and a materials list.
% DIY_PROJECT
{diy_project}

YOUR RESPONSE:
"""
diy_prompt_template = PromptTemplate(input_variables=["diy_project"], template=diy_description_template)

description_chain = LLMChain(llm=llm, prompt=diy_prompt_template)

In [46]:
description_chain

LLMChain(prompt=PromptTemplate(input_variables=['diy_project'], template='Given a DIY project, give a short and simple recipe step-by-step guide on how to complete the project and a materials list.\n% DIY_PROJECT\n{diy_project}\n\nYOUR RESPONSE:\n'), llm=AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001D1E1A0B8D0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001D1E1A74110>, openai_api_key='62855d6dd08945819bf83aee0c104127', openai_proxy='', azure_endpoint='https://dskumar.openai.azure.com/', deployment_name='DskumarDeployment', openai_api_version='2023-07-01-preview', openai_api_type='azure'))

In [38]:
overall_chain = SimpleSequentialChain(chains=[topic_chain, description_chain, verbose=True)

In [47]:
overall_chain

SimpleSequentialChain(verbose=True, chains=[LLMChain(prompt=PromptTemplate(input_variables=['child_description'], template='Your job is to come up with a fun DIY project for the specified gender, age, and description of a kid.\n% CHILD_DESCRIPTION\n{child_description}\n\nYOUR RESPONSE:\n'), llm=AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001D1E1A0B8D0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001D1E1A74110>, openai_api_key='62855d6dd08945819bf83aee0c104127', openai_proxy='', azure_endpoint='https://dskumar.openai.azure.com/', deployment_name='DskumarDeployment', openai_api_version='2023-07-01-preview', openai_api_type='azure')), LLMChain(prompt=PromptTemplate(input_variables=['diy_project'], template='Given a DIY project, give a short and simple recipe step-by-step guide on how to complete the project and a materials list.\n% DIY_PROJECT\n{diy_project}\n\nYOUR RESPONSE:\n'), llm=AzureChatOpenAI(client=<ope

In [52]:
review = overall_chain.run("10-year-old boy")



> Entering new SimpleSequentialChain chain...
DIY Project: Build Your Own Miniature Skate Park

Description:
For the adventurous and creative 10-year-old boy, a fun DIY project would be to build his very own miniature skate park! This project will allow him to unleash his imagination and create a customized skate park that he can use with his skateboards, fingerboards, or toy cars.

Materials Needed:
1. Cardboard or foam board
2. Craft supplies (markers, colored paper, stickers, etc.)
3. Scissors
4. Hot glue gun or craft glue
5. Small wooden blocks or building blocks
6. Small ramps or inclines (can be made from cardboard or foam)
7. Optional: Toy skateboards, fingerboards, or toy cars

Instructions:
1. Start by creating a base for the skate park using a large piece of cardboard or foam board. This will be the foundation on which the ramps and obstacles will be placed.

2. Use the scissors to cut out smaller pieces of cardboard or foam board to create ramps, half-pipes, and other skat

In [53]:
print(review)

Materials Needed:
1. Cardboard or foam board
2. Craft supplies (markers, colored paper, stickers, etc.)
3. Scissors
4. Hot glue gun or craft glue
5. Small wooden blocks or building blocks
6. Small ramps or inclines (can be made from cardboard or foam)
7. Optional: Toy skateboards, fingerboards, or toy cars

Instructions:
1. Create a base for the skate park using a large piece of cardboard or foam board.
2. Cut smaller pieces of cardboard or foam board to create ramps, half-pipes, and other skate park features.
3. Decorate the ramps and obstacles using markers, colored paper, stickers, or other craft supplies.
4. Secure the ramps and obstacles onto the base using a hot glue gun or craft glue.
5. Use small wooden blocks or building blocks to create additional structures like benches, stairs, or rails.
6. Glue the additional structures onto the base or attach them to the ramps for more challenging obstacles.
7. Add any additional details or accessories, such as toy skateboards, fingerboar